In [1]:
import numpy as np
import requests
import re
import pandas as pd
from bs4 import BeautifulSoup
import itertools

In [2]:
url="https://www.flipkart.com/search?q=tv&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off"

In [3]:
data=requests.get(url)
data

<Response [403]>

In [4]:
data.text

'<!DOCTYPE html><html lang=en><meta charset=UTF-8><meta content="width=device-width,initial-scale=1"name=viewport><title>Flipkart reCAPTCHA</title><link href=https://static-assets-web.flixcart.com/batman-returns/batman-returns/s/recaptcha.css rel=stylesheet><script src="https://www.google.com/recaptcha/enterprise.js?render=6Lc49B0pAAAAAIVgOhfwW8i7t7SRO0KSnlSVZRAq"></script><script src=https://static-assets-web.flixcart.com/apex-static/js/recaptcha-enterprise-v1.js async defer></script><div class=container><img alt="Flipkart Logo"class=logo src="https://rukminim1.flixcart.com/www/60/60/promos/14/06/2024/88011666-ce1d-40f0-a8eb-1bac7d164885.png?q=60"><h1 class=header>Are you a human?</h1><p class=subText>Confirming...<div class=loaderContainer><div class=loader></div></div></div>'

In [5]:
soup=BeautifulSoup(data.text)

In [6]:
soup

<!DOCTYPE html>
<html lang="en"><head><meta charset="utf-8"/><meta content="width=device-width,initial-scale=1" name="viewport"/><title>Flipkart reCAPTCHA</title><link href="https://static-assets-web.flixcart.com/batman-returns/batman-returns/s/recaptcha.css" rel="stylesheet"/><script src="https://www.google.com/recaptcha/enterprise.js?render=6Lc49B0pAAAAAIVgOhfwW8i7t7SRO0KSnlSVZRAq"></script><script async="" defer="" src="https://static-assets-web.flixcart.com/apex-static/js/recaptcha-enterprise-v1.js"></script></head><body><div class="container"><img alt="Flipkart Logo" class="logo" src="https://rukminim1.flixcart.com/www/60/60/promos/14/06/2024/88011666-ce1d-40f0-a8eb-1bac7d164885.png?q=60"/><h1 class="header">Are you a human?</h1><p class="subText">Confirming...</p><div class="loaderContainer"><div class="loader"></div></div></div></body></html>

In [10]:
TV_data=soup.find_all("div",class_="KzDlHZ")
TV_data

[]

In [11]:
for i in TV_data:
    print(i.text)

In [12]:
brand_name = []
screen_size_cm = [] 
screen_size_inch = []
resolution = []
screen_type=[]
price = []
ratings=[]
no_of_ratings = []
reviews=[]
launch_year = []

for i in range(1,20):
    
    for i in soup.find_all("div",class_="KzDlHZ"):
        
        if re.findall(r"(^\w+)\s",i.text):
            brand_name.append("".join(re.findall(r"(^\w+)\s",i.text)))
        else:
            brand_name.append(np.nan)

        if re.findall(r"(\d+(?:\.\d+)?)\s*cm\b",i.text):  
            screen_size_cm.append("".join(re.findall(r"(\d+(?:\.\d+)?)\s*cm\b",i.text)))
        else:
            screen_size_cm.append(np.nan)

        if re.findall(r"\((\d+(?:\.\d+)?)\s*inch\)",i.text):
            screen_size_inch.append("".join(re.findall(r"\((\d+(?:\.\d+)?)\s*inch\)",i.text)))
        else:
            screen_size_inch.append(np.nan)

        if re.findall(r"(SD|HD Ready|Full HD|Ultra HD \(4K\)|\(8K\))",i.text):
            resolution.append("".join(re.findall(r"(SD|HD Ready|Full HD|Ultra HD \(4K\)|\(8K\))",i.text)))
        else:
            resolution.append(np.nan)

        if re.findall(r"(LED|OLED|QLED|Mini LED|Laser)",i.text):
            screen_type.append("".join(re.findall(r"(OLED|QLED|Mini LED|LED|Laser)",i.text)))
        else:
            screen_type.append(np.nan)

    for i in soup.find_all("div",class_="Nx9bqj _4b5DiR"):
        
        if re.findall(r"^₹([\d,]+)",i.text):
            price.append("".join(re.findall(r"^₹([\d,]+)",i.text)))
        else:
            price.append(np.nan)

    for i in soup.find_all("div",class_="XQDdHH"):
        
        if re.findall(r"(\d\.\d)",i.text):
            ratings.append("".join(re.findall(r"(\d\.\d)",i.text)))
        else:
            ratings.append(np.nan)

    for i in soup.find_all("span",class_="Wphh3N"):
        
        if re.findall(r"(\d*[,\d*]*)\sRatings",i.text):             #(\d*[,\d*]*)
            no_of_ratings.append("".join(re.findall(r"(\d*[,\d*]*)\sRatings",i.text)))
        else:
            no_of_ratings.append(np.nan)

       
        if re.findall(r"&\s(\d*[,\d*]*)\sReviews",i.text):
            reviews.append("".join(re.findall(r"(\d*[,\d*]*)\sRatings",i.text)))
        else:
            reviews.append(np.nan)

    for i in soup.find_all("div",class_="_6NESgJ"):
        if re.findall(r"\d{4}",i.text):
            launch_year.append("".join(re.findall(r"Launch Year: (\d{4})",i.text)))
        else:
            launch_year.append(np.nan)

In [13]:
# Combine the lists using zip_longest to match the length
data = list(itertools.zip_longest(
    brand_name, screen_size_cm, screen_size_inch, resolution, screen_type,
    price, ratings, no_of_ratings, reviews, launch_year, 
    fillvalue=np.nan  # Fill missing values with NaN
))

In [14]:
df=pd.DataFrame(data,columns=[
    'brand_name', 'screen_size_cm', 'screen_size_inch', 'resolution', 'screen_type',
    'price', 'ratings', 'no_of_ratings', 'reviews',  'launch_year'
])

In [12]:
output_file = "flipkart_edaproject.csv"
df.to_csv(output_file, index=True)